# Tarea 3 MongoDB, arXiv
##### Integrantes:
Bruno Morici, ROL: 202373555-8,
Martin Aranda, ROL: 202373021-1

## Primera Etapa
Instalación de dependencias

In [2]:
# Instalamos dependencias
!pip install pymongo
!pip install tqdm
!pip install json


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: C:\Users\Bruno\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: C:\Users\Bruno\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: C:\Users\Bruno\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import json
from pymongo import MongoClient
from tqdm import tqdm

## Segunda Etapa
Conexión a la BD "arxiv_db" y creación de colección "articles"

In [26]:
# Conexión a Replica Set con preferencia al primario
client = MongoClient("mongodb://mongo1:30001,mongo2:30002,mongo3:30003/?replicaSet=my-replica-set&readPreference=primary&appname=MongoDB%20Compass&ssl=false")

db = client["arxiv_db"]
collection = db["articles"]

try:
    print("✅ Conexión exitosa")
except Exception as e:
    print(f"❌ Error de conexión: {e}")

✅ Conexión exitosa


## Tercera Etapa
Procesamiento del dataset hacia la BD (ejecutar una sola vez, luego los datos quedan en la BD)

Observación: El dataset tiene un JSON en cada línea, no es una lista de JSON's. Recorremos cada JSON, lo parseamos y guardamos en una lista de data (demora unos 6 minutos)

In [27]:
# === Configuración ===
json_file_path = "uarxiv-202730211-2023735558.json"
batch_size = 2000
max_workers = 4  # Ajusta según tu CPU y MongoDB

In [8]:
# === Función para insertar lote ===
def insert_batch(batch):
    try:
        collection.insert_many(batch, ordered=False)
    except Exception as e:
        print("Error al insertar batch:", e)

In [9]:
# === Contar líneas para tqdm ===
def count_lines(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return sum(1 for _ in f)

total_lines = count_lines(json_file_path)

In [28]:
# Cargar y procesar el archivo JSON
print("Iniciando carga de datos a MongoDB...")

# Carga por lotes de 1000 en 1000
def load_data_batch(batch_size=1000):
    batch = []
    total_docs = 0
    
    # Primero contamos las lineas para la barra de progreso
    with open('uarxiv-202730211-2023735558.json', 'r', encoding='utf-8') as f:
        total_lines = sum(1 for _ in f)
    
    # Insertamos los datos en el batch
    with open('uarxiv-202730211-2023735558.json', 'r', encoding='utf-8') as f:
        for line in tqdm(f, total=total_lines, desc="Cargando datos"):
            record = json.loads(line)
            # Agregamos el campo pdf_source
            if "id" in record:
                record["pdf_source"] = f"https://arxiv.org/pdf/{record['id']}"

            batch.append(record)
            
            # Si el batch excede el maximo, lo vaciamos e insertamos los datos en la BD
            if len(batch) >= batch_size:
                collection.insert_many(batch)
                total_docs += len(batch)
                batch = []
        
        # Insertar el ultimo lote, en caso de que el ultimo lote sean menos de 1000
        if batch:
            collection.insert_many(batch)
            total_docs += len(batch)
    
    print(f"\nTotal de documentos insertados: {total_docs}")

load_data_batch() # Llamamos la funcion creada anteriormente

# 3. Verificacion final
print("\nResumen de la base de datos:")
print(f"- Colección: {collection.name}")    
print(f"- Ejemplo de documento: {collection.find_one()}")



Iniciando carga de datos a MongoDB...


Cargando datos:   1%|          | 29999/2744489 [00:07<11:40, 3876.90it/s]


KeyboardInterrupt: 

Consultamos por la carga efectiva de datos en las 3 réplicas

In [29]:
ports = [30001, 30002, 30003]
for i, port in enumerate(ports, start=1):
    client = MongoClient(f"mongodb://mongo{i}:{port}")
    count = client["arxiv_db"]["articles"].estimated_document_count()
    print(f"📦 Documentos estimados en mongo{i}: {count}")

📦 Documentos estimados en mongo1: 30000
📦 Documentos estimados en mongo2: 30000
📦 Documentos estimados en mongo3: 30000


## Cuarta etapa: Consultas

Consulta test

In [30]:
def buscar_software(port, nombre):
    try:
        client = MongoClient(f"mongodb://localhost:{port}", serverSelectionTimeoutMS=5000)
        db = client["arxiv_db"]
        collection = db["articles"]

        resultados = collection.find(
            {"title": {"$regex": "Software", "$options": "i"}},
            {"_id": 0, "title": 1}
        ).limit(3)  # solo para mostrar pocos

        print(f"🟢 {nombre} ({port}) - Artículos con 'Software':")
        encontrados = False
        for doc in resultados:
            print(f"   🔹 {doc['title']}")
            encontrados = True
        if not encontrados:
            print("   ⚠️ No se encontraron artículos.")
        print("-" * 50)
    except Exception as e:
        print(f"🔴 Error en {nombre} ({port}): {e}")

# Ejecutar búsqueda en los tres nodos
buscar_software(30001, "mongo1")
buscar_software(30002, "mongo2")
buscar_software(30003, "mongo3")


🟢 mongo1 (30001) - Artículos con 'Software':
   🔹 Solar Magnetic Tracking. I. Software Comparison and Recommended
  Practices
   🔹 On How Developers Test Open Source Software Systems
   🔹 Experiences of Engineering Grid-Based Medical Software
--------------------------------------------------
🟢 mongo2 (30002) - Artículos con 'Software':
   🔹 Solar Magnetic Tracking. I. Software Comparison and Recommended
  Practices
   🔹 On How Developers Test Open Source Software Systems
   🔹 Experiences of Engineering Grid-Based Medical Software
--------------------------------------------------
🟢 mongo3 (30003) - Artículos con 'Software':
   🔹 Solar Magnetic Tracking. I. Software Comparison and Recommended
  Practices
   🔹 On How Developers Test Open Source Software Systems
   🔹 Experiences of Engineering Grid-Based Medical Software
--------------------------------------------------


Consulta 1

In [ ]:
def leer_articulos_2025(collection):
    try:
        # Defino lo que queremos buscar en base a la filtro del año 2025
        cursor = collection.find(
            {"versions.0.created": {"$regex": r"2025"}},
            {"title": 1, "versions": 1}).limit(20)
        
        art = []

        # Agregamos todo lo recolectado en una lista
        for doc in cursor:
            art.append({
                "title": doc.get("title", "No title"),
                "versions": doc.get("versions", [])
            })
        
        return art
    except Exception as e:
        print(f"Error al leer artículos de 2025: {e}")
        return []
    
resultado = leer_articulos_2025(collection)
i = 1
for elem in resultado:
    print(f"{i}. Título: {elem.get('title', '(sin título)')}")
    primera_version = elem.get('versions', [{}])[0]
    fecha = primera_version.get('created', '¿sin fecha?')
    print(f"   Primera versión (v1): {fecha}")
    print("-" * 50)
    i += 1

Consulta 2

In [15]:
def buscar_articulos_csai_statml_3autores(collection):
    try:
        cursor = collection.find(
            {
                "$or": [
                    {"categories": {"$regex": r"cs\.AI"}},
                    {"categories": {"$regex": r"stat\.ML"}}
                ]
            },
            {"_id": 0, "title": 1, "authors": 1}
        ).limit(100)  # buscamos más para filtrar en Python

        resultados = []

        for doc in cursor:
            authors_str = doc.get("authors", "")
            # separar autores por coma o "and"
            autores = []
            if authors_str:
                # primero separar por comas
                partes = [a.strip() for a in authors_str.split(",")]
                # cada parte puede contener 'and', dividirlas también
                for parte in partes:
                    autores.extend([x.strip() for x in parte.split(" and ") if x.strip() != ""])

            if len(autores) >= 3:
                resultados.append({"title": doc.get("title"), "authors": autores})

            if len(resultados) == 10:
                break

        if resultados:
            print("Artículos en cs.AI o stat.ML con al menos 3 autores:")
            i = 1
            for art in resultados:
                print(f"{i}. Título: {art['title']}")
                print(f"   Autores: {art['authors']}")
                print("-" * 40)
                i += 1
        else:
            print("No se encontraron artículos con al menos 3 autores en las categorías dadas.")

        return resultados

    except Exception as e:
        print(f"Error al buscar artículos: {e}")
        return []

# Ejecutar función
articulos_encontrados = buscar_articulos_csai_statml_3autores(collection)



Artículos en cs.AI o stat.ML con al menos 3 autores:
1. Título: Calculating Valid Domains for BDD-Based Interactive Configuration
   Autores: ['Tarik Hadzic', 'Rune Moller Jensen', 'Henrik Reif Andersen']
----------------------------------------
2. Título: Personalizing Image Search Results on Flickr
   Autores: ['Kristina Lerman', 'Anon Plangprasopchok', 'Chio Wong']
----------------------------------------
3. Título: Unicast and Multicast Qos Routing with Soft Constraint Logic Programming
   Autores: ['Stefano Bistarelli', 'Ugo Montanari', 'Francesca Rossi', 'Francesco Santini']
----------------------------------------
4. Título: A study of structural properties on profiles HMMs
   Autores: ['Juliana S Bernardes', 'Alberto Davila', 'Vitor Santos Costa', 'Gerson\n  Zaverucha']
----------------------------------------
5. Título: Introduction to Arabic Speech Recognition Using CMUSphinx System
   Autores: ['H. Satori', 'M. Harti', 'N. Chenfour']
----------------------------------------


Consulta 3

In [16]:
def leer_hep_ph_con_doi(collection):
    try:
        cursor = collection.find(
            {"categories": "hep-ph", "doi": {"$exists": True, "$ne": ""}},
            {"_id": 0, "title": 1, "categories": 1, "pdf_source": 1}).limit(15)

        resultados = []
        for doc in cursor:
            resultados.append({
                "titulo": doc.get("title", "Sin título"),
                "categorias": doc.get("categories", []),
                "pdf_source": doc.get("pdf_source", "No disponible")
            })

        if resultados:
            print("Artículos en hep-ph con DOI:")
            print()
            i = 1
            for art in resultados:
                print(f"{i}. Título: {art['titulo']}")
                print(f"   Categorías: {art['categorias']}")
                print(f"   PDF: {art['pdf_source']}")
                print("-" * 40)
                i += 1
        else:
            print("No se encontraron artículos en hep-ph con DOI.")

        return resultados

    except Exception as e:
        print(f"Error al buscar artículos hep-ph con DOI: {e}")
        return []

articulos_encontrados = leer_hep_ph_con_doi(collection)

Artículos en hep-ph con DOI:

1. Título: Calculation of prompt diphoton production cross sections at Tevatron and
  LHC energies
   Categorías: hep-ph
   PDF: https://arxiv.org/pdf/0704.0001
----------------------------------------
2. Título: Lifetime of doubly charmed baryons
   Categorías: hep-ph
   PDF: https://arxiv.org/pdf/0704.0016
----------------------------------------
3. Título: Understanding the Flavor Symmetry Breaking and Nucleon Flavor-Spin
  Structure within Chiral Quark Model
   Categorías: hep-ph
   PDF: https://arxiv.org/pdf/0704.0029
----------------------------------------
4. Título: Crystal channeling of LHC forward protons with preserved distribution in
  phase space
   Categorías: hep-ph
   PDF: https://arxiv.org/pdf/0704.0031
----------------------------------------
5. Título: Probing non-standard neutrino interactions with supernova neutrinos
   Categorías: hep-ph
   PDF: https://arxiv.org/pdf/0704.0032
----------------------------------------
6. Título: Experi

Consulta 4

In [17]:
def leer_articulos_con_doi(collection):
    """
    Devuelve títulos, autores y referencia de publicación de artículos con DOI.
    Ordena alfabéticamente por título y limita a 20 resultados.
    """
    try:
        cursor = collection.find(
            {"doi": {"$exists": True, "$ne": ""}},
            {"_id": 0, "title": 1, "authors": 1, "journal-ref": 1}).sort("title", 1).limit(20)

        resultados = []
        for doc in cursor:
            resultados.append({
                "titulo": doc.get("title", "Sin título"),
                "autores": doc.get("authors", []),
                "referencia_publicacion": doc.get("journal-ref", "No disponible")
            })

        if resultados:
            print("Artículos con DOI:")
            print()
            i = 1
            for art in resultados:
                print(f"{i}.Título: {art['titulo']}")
                print(f"  Autores: {art['autores']}")
                print(f"  Referencia publicación: {art['referencia_publicacion']}")
                print("-" * 40)
                i += 1
        else:
            print("No se encontraron artículos con DOI.")

        return resultados

    except Exception as e:
        print(f"Error al buscar artículos con DOI: {e}")
        return []

articulos_encontrados = leer_articulos_con_doi(collection)

Artículos con DOI:

1.Título: "A combinatorial universal $\star$-product" is incorrect
  Autores: Theo Johnson-Freyd
  Referencia publicación: None
----------------------------------------
2.Título: "ATLAS Overview and Main Results"
  Autores: Krzysztof Sliwa
  Referencia publicación: None
----------------------------------------
3.Título: "Ab initio" models of galaxy formation: successes and open problems
  Autores: Gabriella De Lucia
  Referencia publicación: None
----------------------------------------
4.Título: "Actuation at a distance" of microelectromechanical systems using
  photoelectrowetting: proof-of-concept
  Autores: Matthieu Gaudet and Steve Arscott
  Referencia publicación: Appl. Phys. Lett. 100, 224103 (2012)
----------------------------------------
5.Título: "Additivity" versus "Maxitivity" at the heart of the paradoxical and
  efficient nature of Statistics
  Autores: M. R\'emon
  Referencia publicación: None
----------------------------------------
6.Título: "Advanc

Consulta 5

In [18]:
def obtener_articulos_2010_2015(collection):
    try:
        # Expresión regular para años del 2010 al 2015 en la fecha creada
        regex = {"$regex": r"201[0-5]"}
        
        # Solo pedimos los artículos con una primera versión que coincida
        cursor = collection.find(
            { "versions.0.created": regex },
            { "title": 1, "versions": 1 }
        ).limit(15)

        resultados = []
        for doc in cursor:
            title = doc.get("title", "(sin título)")
            versiones = doc.get("versions", [])
            if versiones and "created" in versiones[0]:
                fecha = versiones[0]["created"]
            else:
                fecha = "¿sin fecha?"
                
            resultados.append({
                "title": title,
                "fecha_primera_version": fecha
            })
        i = 1
        for r in resultados:
            print(f"{i}. Titulo: {r['title']}")
            print(f"  Primera versión: {r['fecha_primera_version']}")
            print("-" * 50)
            i += 1

        return resultados

    except Exception as e:
        print(f"Error ejecutando la consulta: {e}")

articulos_encontrados = obtener_articulos_2010_2015(collection)

1. Titulo: A landscape of non-supersymmetric AdS vacua on coset manifolds
  Primera versión: Mon, 4 Jan 2010 13:51:46 GMT
--------------------------------------------------
2. Titulo: Jet Shapes and Jet Algorithms in SCET
  Primera versión: Mon, 4 Jan 2010 20:56:57 GMT
--------------------------------------------------
3. Titulo: A Comprehensive Analysis of Uncertainties Affecting the Stellar Mass -
  Halo Mass Relation for 0<z<4
  Primera versión: Sun, 3 Jan 2010 19:43:29 GMT
--------------------------------------------------
4. Titulo: Testing product states, quantum Merlin-Arthur games and tensor
  optimisation
  Primera versión: Mon, 4 Jan 2010 18:01:41 GMT
--------------------------------------------------
5. Titulo: Mu-Tau Production at Hadron Colliders
  Primera versión: Mon, 4 Jan 2010 04:10:52 GMT
--------------------------------------------------
6. Titulo: New identities involving q-Euler polynomials of higher order
  Primera versión: Mon, 4 Jan 2010 15:34:13 GMT
-----------

Consulta 6

In [19]:
def obtener_articulos_no_nulos(collection):
    try:
        cursor = collection.find(
            # Filtro para artículos que tienen comentarios no nulos.
            {"comments": {"$exists": True, "$ne": None}},
            # Buscamos solo por titulo, comentarios y número de reporte.
            {"title": 1, "comments": 1, "report-no": 1}
        ).sort("updated", -1).limit(10)

        resultados = []
        for doc in cursor:
            resultados.append({
                "title": doc.get("title", "Sin título"),
                "comments": doc.get("comments", []),
                "report_no": doc.get("doi", {}).get("report-no", "null")
            })
        
        return resultados
    
    except Exception as e:
        print(f"Error ejecutando la consulta: {e}")

articulos_encontrados = obtener_articulos_no_nulos(collection)
i = 1
for articulos in articulos_encontrados:
    print(f"{i}. Título: {articulos['title']}")
    print(f"   Comentarios: {articulos['comments']}")
    print(f"   Número de reporte: {articulos['report_no']}")
    print("-" * 50)
    i += 1

1. Título: Computing genus 2 Hilbert-Siegel modular forms over $\Q(\sqrt{5})$ via
  the Jacquet-Langlands correspondence
   Comentarios: 14 pages; title changed; to appear in Experimental Mathematics
   Número de reporte: null
--------------------------------------------------
2. Título: Iterated integral and the loop product
   Comentarios: 18 pages, 1 figure
   Número de reporte: null
--------------------------------------------------
3. Título: Bosonic characters of atomic Cooper pairs across resonance
   Comentarios: 6 pages, 4 figures, accepted by PRA
   Número de reporte: null
--------------------------------------------------
4. Título: Partial cubes: structures, characterizations, and constructions
   Comentarios: 36 pages, 17 figures
   Número de reporte: null
--------------------------------------------------
5. Título: Polymer Quantum Mechanics and its Continuum Limit
   Comentarios: 16 pages, no figures. Typos corrected to match published version
   Número de reporte: null


Código para asegurar consistencia de los datos

Insertamos un dato "test" en el contenedor principal.

In [20]:
#Con el siguiente código insertamos un documento de prueba en el nodo primario
def insertar_en_nodoPrimario(collection, data):
    try:
        result = collection.insert_one(data)
        print(f"Documento insertado en mongo1 con ID: {result.inserted_id}")
    except Exception as e:
        print(f"Error al insertar en mongo1: {e}")

insertar_en_nodoPrimario(collection, {"test": "replicacion_OK"})

Documento insertado en mongo1 con ID: 684b83ffa546dbb551bd9e89


Ahora, consultamos en los tres contenedores si se encuentra el dato "test".

In [21]:
#Ahora buscamos el dato insertado en cada nodo del Replica Set
def buscar_dato_test_por_nodo(port, contenedor):
    try:
        client = MongoClient(f"mongodb://localhost:{port}", serverSelectionTimeoutMS=5000)
        db = client["arxiv_db"]
        collection = db["articles"]

        result = collection.find_one({"test": "replicacion_OK"})
        if result:
            print(f"-> Dato encontrado en {contenedor}: {result}")
        else:
            print(f"-> Dato no encontrado en {contenedor}.")
    except Exception as e:
        print(f"Error al buscar en {contenedor}: {e}")

#Llamamos por cada nodo si existe el dato insertado
buscar_dato_test_por_nodo(30001, "mongo1")
buscar_dato_test_por_nodo(30002, "mongo2")
buscar_dato_test_por_nodo(30003, "mongo3")

-> Dato encontrado en mongo1: {'_id': ObjectId('684b83ffa546dbb551bd9e89'), 'test': 'replicacion_OK'}
-> Dato encontrado en mongo2: {'_id': ObjectId('684b83ffa546dbb551bd9e89'), 'test': 'replicacion_OK'}
-> Dato encontrado en mongo3: {'_id': ObjectId('684b83ffa546dbb551bd9e89'), 'test': 'replicacion_OK'}


En el siguiente paso, actualizamos el dato en el nodo principal y verificamos la replicación en los otros nodos.

In [22]:
def actualizar_dato_test():
    try:
        # Actualizamos el campo "test" a "Actualizado!" en el nodo primario
        result = collection.update_one(
            {"test": "replicacion_OK"},
            {"$set": {"test": "Actualizado!"}}
        )
        if result.modified_count > 0:
            print("-> Dato actualizado correctamente.")
        else:
            print("-> No se encontró el dato para actualizar.")
    except Exception as e:
        print(f"Error al actualizar el dato: {e}")

def buscar_dato_test_por_nodo(port, contenedor):
    try:
        client = MongoClient(f"mongodb://localhost:{port}", serverSelectionTimeoutMS=5000)
        db = client["arxiv_db"]
        collection = db["articles"]

        result = collection.find_one({"test": "Actualizado!"})
        if result:
            print(f"-> Dato encontrado en {contenedor}: {result}")
        else:
            print(f"-> Dato no encontrado en {contenedor}.")
    except Exception as e:
        print(f"Error al buscar en {contenedor}: {e}")

actualizar_dato_test()
print("-"*50)
buscar_dato_test_por_nodo(30001, "mongo1")
buscar_dato_test_por_nodo(30002, "mongo2")
buscar_dato_test_por_nodo(30003, "mongo3")

-> Dato actualizado correctamente.
--------------------------------------------------
-> Dato encontrado en mongo1: {'_id': ObjectId('684b83ffa546dbb551bd9e89'), 'test': 'Actualizado!'}
-> Dato encontrado en mongo2: {'_id': ObjectId('684b83ffa546dbb551bd9e89'), 'test': 'Actualizado!'}
-> Dato encontrado en mongo3: {'_id': ObjectId('684b83ffa546dbb551bd9e89'), 'test': 'Actualizado!'}


Ahora borramos el dato que hemos actualizado y lo buscamos en los 3 contenedores para verificar efectivamente la consistencia de los datos

In [23]:
def borrar_dato_test():
    try:
        result = collection.delete_one({"test": "Actualizado!"})
        if result.deleted_count > 0:
            print("-> Dato borrado correctamente.")
        else:
            print("-> No se encontró el dato para borrar.")
    except Exception as e:
        print(f"Error al borrar el dato: {e}")

def buscar_dato_test_por_nodo(port, contenedor):
    try:
        client = MongoClient(f"mongodb://localhost:{port}", serverSelectionTimeoutMS=5000)
        db = client["arxiv_db"]
        collection = db["articles"]

        result = collection.find_one({"test": "Actualizado!"})
        if result:
            print(f"-> Dato encontrado en {contenedor}: {result}")
        else:
            print(f"-> Dato no encontrado en {contenedor}.")
    except Exception as e:
        print(f"Error al buscar en {contenedor}: {e}")

borrar_dato_test()
print("-"*50)
buscar_dato_test_por_nodo(30001, "mongo1")
buscar_dato_test_por_nodo(30002, "mongo2")
buscar_dato_test_por_nodo(30003, "mongo3")

-> Dato borrado correctamente.
--------------------------------------------------
-> Dato no encontrado en mongo1.
-> Dato no encontrado en mongo2.
-> Dato no encontrado en mongo3.
